- OBJECTIVE 
        Extract all individual information about hotel of a particular city i.e. Bangalore.
        

- TASK GIVEN
        Make a scrapper that visits all the individual pages & then on each
        page visits individual hotel then extracts information.
    
- TOOLS USED
        Beautiful soup, Selenium
        
        
- PROGRAMMING LANGUAGE
        Python

In [28]:
import requests
import time
import lxml
from selenium import webdriver
from bs4 import BeautifulSoup
import data
import random
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from urllib.request import Request,urlopen
from bs4 import BeautifulSoup as soup
import json

In [2]:
baseURL = "https://me.cleartrip.com/"
cityURL = "https://me.cleartrip.com/hotels/india/bangalore"

In [7]:
baseURL = data.baseURL
cityURL = data.cityURL

In [8]:
page = requests.get(cityURL)
soup = BeautifulSoup(page.text, 'html.parser')

In [11]:
pageNumber = soup.select('.pagination a')
pageNumber = pageNumber[:-1]
print("Total Number of Pages :", len(pageNumber) + 1)

Total Number of Pages : 43


In [14]:
for i in pageNumber:
    pageNumberLinks = []
    pageNumberLinks = (baseURL + i.get('href'))
    with open("PageNumberLinks.txt", 'a') as file:
        file.write(pageNumberLinks + "\n")

In [23]:
baseURL = data.baseURL
cityURL = data.cityURL

cityURLS = []
with open ("PageNumberLinks.txt", 'r') as file:
    l = file.readlines()
    cityURLS = []
    cityURLS.append(cityURL)
    for i in l:
        cityURLS.append(i.strip())
    
print(cityURLS)

['https://me.cleartrip.com/hotels/india/bangalore', 'https://me.cleartrip.com//hotels/india/bangalore?page=2', 'https://me.cleartrip.com//hotels/india/bangalore?page=3', 'https://me.cleartrip.com//hotels/india/bangalore?page=4', 'https://me.cleartrip.com//hotels/india/bangalore?page=5', 'https://me.cleartrip.com//hotels/india/bangalore?page=6', 'https://me.cleartrip.com//hotels/india/bangalore?page=7', 'https://me.cleartrip.com//hotels/india/bangalore?page=8', 'https://me.cleartrip.com//hotels/india/bangalore?page=9', 'https://me.cleartrip.com//hotels/india/bangalore?page=10', 'https://me.cleartrip.com//hotels/india/bangalore?page=11', 'https://me.cleartrip.com//hotels/india/bangalore?page=12', 'https://me.cleartrip.com//hotels/india/bangalore?page=13', 'https://me.cleartrip.com//hotels/india/bangalore?page=14', 'https://me.cleartrip.com//hotels/india/bangalore?page=15', 'https://me.cleartrip.com//hotels/india/bangalore?page=16', 'https://me.cleartrip.com//hotels/india/bangalore?page=1

In [24]:
res = requests.get(cityURL)
soup = BeautifulSoup(res.text, 'lxml')

In [25]:
for url in cityURLS:
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    hotelRaw = soup.select('#hotelsList a')
    for i in hotelRaw:
        if i != None:
            hotelsURL = (baseURL + str(i.get('href')))
            
            with open ("HotelNotCleaned.txt", 'a') as file:
                file.write(hotelsURL + "\n")

In [26]:
with open("HotelNotCleaned.txt", 'r') as file:
    lines = file.readlines()
    
with open("HotelLinks.txt", "a") as file:
    for line in lines:
        if (len(line) > 30):
            file.write(line)
            
os.remove("HotelNotCleaned.txt")            

In [29]:
def download_hotel_data(hotel_url):
    url_client = urlopen(hotel_url)
    page_html = url_client.read()
    page_soup=soup(page_html,'html.parser')
    url_client.close()
    return page_soup
def find_name(page_soup):
    try: 
        hotel_name=page_soup.title
        hotel_name=hotel_name.text
        hotel_name=hotel_name.split(',')[0]
       
        return hotel_name
    except:
        return None
def hotel_address(page_soup):
    try:
        hotel_address=page_soup.title
        hotel_address=hotel_address.text
        hotel_address=hotel_address.split(',')[1]
        return hotel_address
    except:
        return None
def hotel_price(page_soup):
    try:
        hotel_price=page_soup.find('p',{'class':'minprice'}).contents[-1]
        return hotel_price
    except:
        return None
def hotel_rating(page_soup):
    try:
        rating=page_soup.find(class_="inline row trustBadges")
        hotel_tarating=rating.find('small').text
        return hotel_tarating
    except:
        return None
def hotel_dscr(page_soup):
    data=""
    try:
        abouthotel=page_soup.find('div', class_='amenitiesCategory')
        about=abouthotel.find_all('p')
        for p in about:
            data = data + p.text
        return data
    except:
        return None
def hotel_others(page_soup):
    dct={}
    others=page_soup.find_all('div',{'class':'amenities-category'})
    tstst = [i for i in others]
    try:
        check_in=tstst[0].text.split(" ")[5]# if tstst[0].text.split(" ")[5] else 0
        check_out=tstst[0].text.split(" ")[9]
        rooms_no=tstst[0].text.split(" ")[13]# if tstst[0].text.split(" ")[13] else 0
        dct['check_in']=check_in
        dct['check_out']=check_out
        dct['rooms']=rooms_no
        others=tstst[1]
        heads=[i.text for i in others.find_all('div')]
        lsts=others.findAll('ul',{'class':'list-inline amenities'})
        lk=0
        for i in lsts:
            key=heads[lk].lower()
            key=re.sub('[^ a-zA-Z0-9]','',key)
            key='_'.join(key.split(' '))
            dct[key]=i.text.split("  ")[1:]
            lk+=1
        return dct
    except:
        return None
def create_dict(url):
    page_soup=download_hotel_data(url)
    dct=dict()
    dct['hotel_name']=find_name(page_soup)
    dct['hotel_url']=str(url)
    dct['hotel_address']=hotel_address(page_soup)
    dct['hotel_rating']=hotel_rating(page_soup)
    dct['hotel_price']=hotel_price(page_soup)
    dct['hotel_info']=hotel_dscr(page_soup)
    others=hotel_others(page_soup)
    try:
        for key in others:
            dct[key]=others[key]
        return dct
    except:
        return dct

In [30]:
# download_hotel_data("https://www.cleartrip.com/hotels/india/bangalore")
# print(find_name())

In [31]:
with open ("HotelLinks.txt", 'r') as file:
    l = file.readlines()
    allHotels = []
    for i in l:
        allHotels.append(i.strip())

In [32]:
ls_test = []
for url in allHotels:
    # print(url)
    ls_test.append(create_dict(url))
# print(ls_test)

In [33]:
with open('processed_data.json', 'w') as fp:
    json.dump(ls_test, fp)

In [35]:
import pandas as pd
df = pd.DataFrame(ls_test)

In [2]:
#df.to_csv('processed_dataframe.csv')
a = pd.read_csv('processed_dataframe.csv')

<IPython.core.display.Javascript object>

In [5]:
a['hotel_rating'].value_counts()

3 star    537
2 star    123
4 star     97
5 star     48
1 star     30
OFF         7
Name: hotel_rating, dtype: int64